In [14]:
from datetime import datetime, timedelta
import time
import random
import pandas as pd
pd.set_option("display.max_rows", None, "display.max_columns", None)
import datetime
import mysql.connector
from kafka import KafkaProducer
import json
from cassandra.cqlengine.models import  Model
import cassandra

In [15]:
host = 'localhost'
port = '3306'
db_name = 'Data_Warehouse'
user = 'root'
password = '1'
url = 'jdbc:mysql://' + host + ':' + port + '/' + db_name
driver = "com.mysql.cj.jdbc.Driver"

kafka_bootstrap_severs = "localhost:9092"
producer = KafkaProducer(bootstrap_servers= kafka_bootstrap_severs, value_serializer=lambda v: json.dumps(v).encode('utf-8'))
kafka_topic = "test2"


In [16]:

def get_data_from_job(user,password,host,database):
    cnx = mysql.connector.connect(user=user, password=password,
                                         host=host,
                                      database=database)
    query = """select id as job_id,campaign_id , group_id , company_id from job"""
    mysql_data = pd.read_sql(query,cnx)
    return mysql_data

def get_data_from_publisher(user,password,host,database):
    cnx = mysql.connector.connect(user=user, password=password,
                                         host=host,
                                      database=database)
    query = """select distinct(id) as publisher_id from master_publisher"""
    mysql_data = pd.read_sql(query,cnx)
    return mysql_data


def generating_dummy_data(n_records):
    publisher = get_data_from_publisher(user,password,host,db_name)
    publisher = publisher['publisher_id'].to_list()
    jobs_data = get_data_from_job(user,password,host,db_name)
    job_list = jobs_data['job_id'].to_list()
    campaign_list = jobs_data['campaign_id'].to_list()
    company_list = jobs_data['company_id'].to_list()
    group_list = jobs_data[jobs_data['group_id'].notnull()]['group_id'].astype(int).to_list()
    i = 0 
    fake_records = n_records
    while i <= fake_records:
        create_time = str(cassandra.util.uuid_from_time(datetime.datetime.now()))
        bid = random.randint(0,1)
        interact = ['click','conversion','qualified','unqualified']
        custom_track = random.choices(interact,weights=(70,10,10,10))[0]
        job_id = random.choice(job_list)
        publisher_id = random.choice(publisher)
        group_id = random.choice(group_list)
        campaign_id = random.choice(campaign_list)
        ts = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        # sql = """ INSERT INTO tracking (create_time,bid,campaign_id,custom_track,group_id,job_id,publisher_id,ts) VALUES ('{}',{},{},'{}',{},{},{},'{}')""".format(create_time,bid,campaign_id,custom_track,group_id,job_id,publisher_id,ts)
        data = {
            "create_time": create_time,
            "bid":bid,
            "campaign_id":campaign_id,
            "custom_track":custom_track,
            "group_id":group_id,
            "job_id":job_id,
            "publisher_id":publisher_id,
            "ts":ts,
        }
        print(data)
        producer.send(kafka_topic, value=data)
        # print(sql)
        # session.execute(sql)
        i+=1 
    return print("Data Generated Successfully")


In [17]:
status = "ON"
while status == "ON":
    generating_dummy_data(n_records = random.randint(1,5))
    time.sleep(1)
producer.close()

C:\Users\ACER\AppData\Local\Temp\ipykernel_16640\2486201334.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  mysql_data = pd.read_sql(query,cnx)
C:\Users\ACER\AppData\Local\Temp\ipykernel_16640\2486201334.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  mysql_data = pd.read_sql(query,cnx)


{'create_time': '3362bc4a-30d3-11ef-b294-46d5fe7aeebf', 'bid': 1, 'campaign_id': 223, 'custom_track': 'click', 'group_id': 30, 'job_id': 1840, 'publisher_id': 13, 'ts': '2024-06-22 20:08:33'}
{'create_time': '3362e38c-30d3-11ef-8a9f-b2cc74031a55', 'bid': 1, 'campaign_id': 144, 'custom_track': 'click', 'group_id': 27, 'job_id': 807, 'publisher_id': 22, 'ts': '2024-06-22 20:08:33'}
{'create_time': '3363317a-30d3-11ef-b04e-7b23c1af7dc2', 'bid': 0, 'campaign_id': 61, 'custom_track': 'conversion', 'group_id': 22, 'job_id': 1587, 'publisher_id': 15, 'ts': '2024-06-22 20:08:33'}
{'create_time': '3363317a-30d3-11ef-93fb-15c2551c8ea3', 'bid': 1, 'campaign_id': 12, 'custom_track': 'unqualified', 'group_id': 16, 'job_id': 261, 'publisher_id': 31, 'ts': '2024-06-22 20:08:33'}
{'create_time': '3363317a-30d3-11ef-a59a-02babc643d29', 'bid': 1, 'campaign_id': 5, 'custom_track': 'click', 'group_id': 10, 'job_id': 1375, 'publisher_id': 36, 'ts': '2024-06-22 20:08:33'}
{'create_time': '3363317a-30d3-11ef

C:\Users\ACER\AppData\Local\Temp\ipykernel_16640\2486201334.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  mysql_data = pd.read_sql(query,cnx)
C:\Users\ACER\AppData\Local\Temp\ipykernel_16640\2486201334.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  mysql_data = pd.read_sql(query,cnx)


{'create_time': '346cfede-30d3-11ef-89b2-5891dfa7fac0', 'bid': 0, 'campaign_id': 61, 'custom_track': 'click', 'group_id': 41, 'job_id': 304, 'publisher_id': 1, 'ts': '2024-06-22 20:08:34'}
{'create_time': '346d145a-30d3-11ef-9a31-19ee26bb3d8a', 'bid': 1, 'campaign_id': 1, 'custom_track': 'qualified', 'group_id': 10, 'job_id': 1064, 'publisher_id': 15, 'ts': '2024-06-22 20:08:34'}
{'create_time': '346d28d2-30d3-11ef-a8b4-a8153ecc7400', 'bid': 1, 'campaign_id': 59, 'custom_track': 'qualified', 'group_id': 25, 'job_id': 739, 'publisher_id': 11, 'ts': '2024-06-22 20:08:34'}
Data Generated Successfully


C:\Users\ACER\AppData\Local\Temp\ipykernel_16640\2486201334.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  mysql_data = pd.read_sql(query,cnx)
C:\Users\ACER\AppData\Local\Temp\ipykernel_16640\2486201334.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  mysql_data = pd.read_sql(query,cnx)


...

C:\Users\ACER\AppData\Local\Temp\ipykernel_16640\2486201334.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  mysql_data = pd.read_sql(query,cnx)
C:\Users\ACER\AppData\Local\Temp\ipykernel_16640\2486201334.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  mysql_data = pd.read_sql(query,cnx)


{'create_time': '37b06dce-30d3-11ef-8927-6b4d77bb621f', 'bid': 0, 'campaign_id': 61, 'custom_track': 'click', 'group_id': 17, 'job_id': 655, 'publisher_id': 28, 'ts': '2024-06-22 20:08:40'}
{'create_time': '37b094b6-30d3-11ef-8883-80681473a707', 'bid': 0, 'campaign_id': 1, 'custom_track': 'click', 'group_id': 31, 'job_id': 1281, 'publisher_id': 16, 'ts': '2024-06-22 20:08:40'}
{'create_time': '37b094b6-30d3-11ef-8312-bcc278c8cb08', 'bid': 0, 'campaign_id': 1, 'custom_track': 'click', 'group_id': 10, 'job_id': 338, 'publisher_id': 31, 'ts': '2024-06-22 20:08:40'}
{'create_time': '37b094b6-30d3-11ef-b449-599d52200290', 'bid': 0, 'campaign_id': 48, 'custom_track': 'click', 'group_id': 10, 'job_id': 1221, 'publisher_id': 27, 'ts': '2024-06-22 20:08:40'}
{'create_time': '37b094b6-30d3-11ef-b482-3f64eb85eb78', 'bid': 1, 'campaign_id': 10, 'custom_track': 'click', 'group_id': 32, 'job_id': 86, 'publisher_id': 1, 'ts': '2024-06-22 20:08:40'}
Data Generated Successfully


C:\Users\ACER\AppData\Local\Temp\ipykernel_16640\2486201334.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  mysql_data = pd.read_sql(query,cnx)
C:\Users\ACER\AppData\Local\Temp\ipykernel_16640\2486201334.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  mysql_data = pd.read_sql(query,cnx)


{'create_time': '38b8f1f0-30d3-11ef-b2f7-ef16f1a79b84', 'bid': 0, 'campaign_id': 221, 'custom_track': 'click', 'group_id': 11, 'job_id': 1499, 'publisher_id': 15, 'ts': '2024-06-22 20:08:41'}
{'create_time': '38b918f6-30d3-11ef-9a1e-55dd052258fc', 'bid': 1, 'campaign_id': 144, 'custom_track': 'click', 'group_id': 11, 'job_id': 85, 'publisher_id': 22, 'ts': '2024-06-22 20:08:41'}
{'create_time': '38b918f6-30d3-11ef-84c0-c04607bc5690', 'bid': 1, 'campaign_id': 4, 'custom_track': 'qualified', 'group_id': 15, 'job_id': 696, 'publisher_id': 30, 'ts': '2024-06-22 20:08:41'}
Data Generated Successfully


KeyboardInterrupt: 